In [9]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
import os
import itertools
from dask.diagnostics import ProgressBar
from dask.distributed import Client
from ExperimentTracker2 import PhaseOneExperimentTracker
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from lightgbm import LGBMClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from dask_ml.model_selection import train_test_split

# Initialize Dask client for parallel processing
# client = Client()

In [10]:
# Load large dataset with Dask
df = dd.read_csv("data/merged_data.csv")

In [11]:
# Convert data types for memory efficiency
def convert_dtypes(df):
    df['order_value'] = df['order_value'].astype('float32')
    df['refund_value'] = df['refund_value'].astype('float32')
    df['num_items_ordered'] = df['num_items_ordered'].astype(float).round().astype('uint8')
    df['order_date'] = dd.to_datetime(df['order_date'])
    df['first_order_datetime'] = dd.to_datetime(df['first_order_datetime'])
    df[['country_code', 'collect_type', 'payment_method']] = df[['country_code', 'collect_type', 'payment_method']].astype('category')
    return df

df = convert_dtypes(df)

In [12]:
# Payment method grouping
def group_payment_methods(payment_method):
    mapping = {
        'CreditCard': ['GenericCreditCard', 'CybersourceCreditCard', 'CybersourceApplePay', 'CreditCard'],
        'DigitalWallet': ['GCash', 'AFbKash', 'JazzCashWallet', 'AdyenBoost', 'PayPal'],
        'BankTransfer': ['XenditDirectDebit', 'RazerOnlineBanking'],
        'PaymentOnDelivery': ['Invoice', 'PayOnDelivery']
    }
    for key, values in mapping.items():
        if payment_method in values:
            return key
    return 'Others'

df['payment_method'] = df['payment_method'].map(group_payment_methods)

c:\Users\yongj\env\Lib\site-packages\dask_expr\_collection.py:4196: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('payment_method', 'category'))

  warnings.warn(meta_warning(meta))


In [13]:
# Date transformations
def date_transformations(df):
    df['days_since_first_order'] = (df['order_date'] - df['first_order_datetime']).dt.days
    df = df.drop(columns=['first_order_datetime'])
    df['order_date_day_of_week'] = df['order_date'].dt.dayofweek
    df['order_date_day'] = df['order_date'].dt.day
    df['order_date_month'] = df['order_date'].dt.month
    df['order_date_year'] = df['order_date'].dt.year
    df = df.drop(columns=['order_date'])
    return df

df = date_transformations(df)
df = df.drop(columns=['order_id', 'customer_id'])

In [14]:
# Split data
X = df.drop(columns=['is_fraud'])
y = df['is_fraud']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


c:\Users\yongj\env\Lib\site-packages\dask_ml\model_selection\_split.py:464: FutureWarning: The default value for 'shuffle' must be specified when splitting DataFrames. In the future DataFrames will automatically be shuffled within blocks prior to splitting. Specify 'shuffle=True' to adopt the future behavior now, or 'shuffle=False' to retain the previous behavior.
  warnings.warn(


In [15]:
# XGBoost
from xgboost import XGBClassifier

In [16]:
# Define experiment configurations
search_space = {
    'scaler': [None, StandardScaler(), MinMaxScaler(), RobustScaler()],
    'encode': [{'apply': True, 'columns': ['categorical_col']}],
    'models': [
        # {'name': 'LogisticRegression', 'instance': LogisticRegression()},
        # {'name': 'RandomForest', 'instance': RandomForestClassifier()},
        # {'name': 'LightGBM', 'instance': LGBMClassifier()},
        # {'name': 'GaussianNB', 'instance': GaussianNB()},
        # {'name': 'DecisionTree', 'instance': DecisionTreeClassifier()},
        # {'name': 'GradientBoosting', 'instance': GradientBoostingClassifier()},
        # {'name': 'CatBoostClassifier', 'instance': CatBoostClassifier()},
        # xg boost
        {'name': 'XGBoost', 'instance': XGBClassifier()},
    ]
}

# Generate all combinations
keys, values = zip(*search_space.items())
experiment_combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

categorical_cols = ['payment_method', 'country_code', 'collect_type']
numeric_columns = ['order_value', 'refund_value', 'num_items_ordered', 'days_since_first_order',
                   'order_date_day_of_week', 'order_date_day', 'order_date_month', 'order_date_year']

In [17]:
print(len(experiment_combinations))

4


In [18]:
print(experiment_combinations,X_train.shape,y_train.shape,X_test.shape,y_test.shape,numeric_columns,categorical_cols)

[{'scaler': None, 'encode': {'apply': True, 'columns': ['categorical_col']}, 'models': {'name': 'XGBoost', 'instance': XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)}}, {'scaler': StandardScaler(), 'encode': {'apply': True, 'columns': ['categorical_col']}, 'models': {'name': 'XGBoost', 'i

In [19]:
from ExperimentTracker2 import PhaseOneExperimentTracker

In [20]:
# client = Client(local_directory="dask_tmp")

# X_train_future = client.scatter(X_train, broadcast=True)
# y_train_future = client.scatter(y_train, broadcast=True)
# X_test_future = client.scatter(X_test, broadcast=True)
# y_test_future = client.scatter(y_test, broadcast=True)

In [22]:
# Initialize tracker and run experiments
tracker = PhaseOneExperimentTracker("Phase-1 (Final)")
tracker.run_experiments(experiment_combinations, X_train, y_train, X_test, y_test, numeric_columns, categorical_cols)

c:\Users\yongj\env\Lib\site-packages\distributed\node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 56675 instead
  warnings.warn(


Starting run: XG_NoneType_Enc_202502132111


c:\Users\yongj\OneDrive - Singapore Polytechnic\SP DAAA-Y2\Y2S2\PAI\PAI_CA2\ExperimentTracker2.py:156: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  pipeline.fit(X_train.compute(), y_train.compute().ravel())
c:\Users\yongj\env\Lib\site-packages\mlflow\types\utils.py:435: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.htm

Completed run: XG_NoneType_Enc_202502132111
🏃 View run XG_NoneType_Enc_202502132111 at: https://dagshub.com/REHXZ/PAI_CA2.mlflow/#/experiments/18/runs/f4ba821503dc4f89a33a859f0e88b0c6
🧪 View experiment at: https://dagshub.com/REHXZ/PAI_CA2.mlflow/#/experiments/18
end run
Starting run: XG_Standard_Enc_202502132119


c:\Users\yongj\OneDrive - Singapore Polytechnic\SP DAAA-Y2\Y2S2\PAI\PAI_CA2\ExperimentTracker2.py:156: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  pipeline.fit(X_train.compute(), y_train.compute().ravel())
c:\Users\yongj\env\Lib\site-packages\mlflow\types\utils.py:435: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.htm

...

Completed run: XG_Standard_Enc_202502132119
🏃 View run XG_Standard_Enc_202502132119 at: https://dagshub.com/REHXZ/PAI_CA2.mlflow/#/experiments/18/runs/e0e6c8b301df4eefac333b775077fc62
🧪 View experiment at: https://dagshub.com/REHXZ/PAI_CA2.mlflow/#/experiments/18
end run
Starting run: XG_MinMax_Enc_202502132126


c:\Users\yongj\OneDrive - Singapore Polytechnic\SP DAAA-Y2\Y2S2\PAI\PAI_CA2\ExperimentTracker2.py:156: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  pipeline.fit(X_train.compute(), y_train.compute().ravel())
c:\Users\yongj\env\Lib\site-packages\mlflow\types\utils.py:435: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.htm

2025-02-13 21:31:35,055 - distributed.scheduler - ERROR - 
Traceback (most recent call last):
  File "c:\Users\yongj\env\Lib\site-packages\distributed\compatibility.py", line 204, in asyncio_run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\yongj\AppData\Local\Programs\Python\Python311\Lib\asyncio\runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\yongj\AppData\Local\Programs\Python\Python311\Lib\asyncio\base_events.py", line 640, in run_until_complete
    self.run_forever()
  File "C:\Users\yongj\AppData\Local\Programs\Python\Python311\Lib\asyncio\base_events.py", line 607, in run_forever
    self._run_once()
  File "C:\Users\yongj\AppData\Local\Programs\Python\Python311\Lib\asyncio\base_events.py", line 1884, in _run_once
    event_list = self._selector.select(timeout)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\yongj\AppData\Local\Programs\Python\

🏃 View run XG_MinMax_Enc_202502132126 at: https://dagshub.com/REHXZ/PAI_CA2.mlflow/#/experiments/18/runs/03b028a30c6a4aebb49993b519d00e14
🧪 View experiment at: https://dagshub.com/REHXZ/PAI_CA2.mlflow/#/experiments/18
Error in run XG_MinMax_Enc_202502132126: Could not pickle the task to send it to the workers.
end run
Starting run: XG_Robust_Enc_202502132131
🏃 View run XG_Robust_Enc_202502132131 at: https://dagshub.com/REHXZ/PAI_CA2.mlflow/#/experiments/18/runs/88a1423f45484ee68c906e33bac5883c
🧪 View experiment at: https://dagshub.com/REHXZ/PAI_CA2.mlflow/#/experiments/18
Error in run XG_Robust_Enc_202502132131: IOLoop is closed
end run


RuntimeError: IOLoop is closed

---

In [ ]:
# Load datasets
X_train = dd.read_csv('./data/X_train.csv')
y_train = dd.read_csv('./data/y_train.csv')

X_test = dd.read_csv('./data/X_test.csv')
y_test = dd.read_csv('./data/y_test.csv')

X_train_ISO = dd.read_csv('./data/X_train_ISO.csv')
y_train_ISO = dd.read_csv('./data/y_train_ISO.csv')

X_train_ISO_SMOTE = dd.read_csv('./data/X_train_ISO_smote.csv')
y_train_ISO_SMOTE = dd.read_csv('./data/y_train_ISO_smote.csv')

X_train_ISO_ROS = dd.read_csv('./data/X_train_ISO_ros.csv')
y_train_ISO_ROS = dd.read_csv('./data/y_train_ISO_ros.csv')

X_train_ISO_RUS = dd.read_csv('./data/X_train_ISO_rus.csv')
y_train_ISO_RUS = dd.read_csv('./data/y_train_ISO_rus.csv')

X_train_LOF = dd.read_csv('./data/X_train_LOF.csv')
y_train_LOF = dd.read_csv('./data/y_train_LOF.csv')

X_train_LOF_SMOTE = dd.read_csv('./data/X_train_LOF_smote.csv')
y_train_LOF_SMOTE = dd.read_csv('./data/y_train_LOF_smote.csv')

X_train_LOF_ROS = dd.read_csv('./data/X_train_LOF_ros.csv')
y_train_LOF_ROS = dd.read_csv('./data/y_train_LOF_ros.csv')

X_train_LOF_RUS = dd.read_csv('./data/X_train_LOF_rus.csv')
y_train_LOF_RUS = dd.read_csv('./data/y_train_LOF_rus.csv')

X_train_smote = dd.read_csv('./data/X_train_smote.csv')
y_train_smote = dd.read_csv('./data/y_train_smote.csv')

X_train_ros = dd.read_csv('./data/X_train_ros.csv')
y_train_ros = dd.read_csv('./data/y_train_ros.csv')

X_train_rus = dd.read_csv('./data/X_train_rus.csv')
y_train_rus = dd.read_csv('./data/y_train_rus.csv')

In [15]:
datasets = [
    ("dataset_default", X_train, y_train),
    ("dataset_ISO", X_train_ISO, y_train_ISO),
    ("dataset_ISO_SMOTE", X_train_ISO_SMOTE, y_train_ISO_SMOTE),
    ("dataset_ISO_ROS", X_train_ISO_ROS, y_train_ISO_ROS),
    ("dataset_ISO_RUS", X_train_ISO_RUS, y_train_ISO_RUS),
    ("dataset_LOF", X_train_LOF, y_train_LOF),
    ("dataset_LOF_SMOTE", X_train_LOF_SMOTE, y_train_LOF_SMOTE),
    ("dataset_LOF_ROS", X_train_LOF_ROS, y_train_LOF_ROS),
    ("dataset_LOF_RUS", X_train_LOF_RUS, y_train_LOF_RUS),
    ("dataset_SMOTE", X_train_smote, y_train_smote),
    ("dataset_ROS", X_train_ros, y_train_ros),
    ("dataset_RUS", X_train_rus, y_train_rus)
]

In [16]:
# Define experiment configurations
search_space = {
    'scaler': [None],
    'encode': [{'apply': True, 'columns': ['categorical_col']}],
    'models': [
        {'name': 'LogisticRegression', 'instance': LogisticRegression()},
        {'name': 'RandomForest', 'instance': RandomForestClassifier()},
        {'name': 'LightGBM', 'instance': LGBMClassifier()},
        {'name': 'GaussianNB', 'instance': GaussianNB()},
        {'name': 'DecisionTree', 'instance': DecisionTreeClassifier()},
        {'name': 'GradientBoosting', 'instance': GradientBoostingClassifier()},
    ]
}

# Generate all combinations
keys, values = zip(*search_space.items())
experiment_combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

categorical_cols = ['payment_method', 'country_code', 'collect_type']
numeric_columns = ['order_value', 'refund_value', 'num_items_ordered', 'days_since_first_order',
                   'order_date_day_of_week', 'order_date_day', 'order_date_month', 'order_date_year']

In [17]:
len(experiment_combinations)

6

In [18]:
# # Initialize the tracker
# from ExperimentTracker2 import PhaseTwoExperimentTracker
# tracker = PhaseTwoExperimentTracker("Phase-2 (Final)")

# # Load checkpoint file
# tracker.completed_runs

# # Run experiments with checkpointing
# tracker.run_experiments(
#     datasets=datasets,
#     experiment_combinations=experiment_combinations,
#     X_test=X_test,
#     y_test=y_test,
#     numeric_columns=numeric_columns,
#     categorical_cols=categorical_cols
# )

In [19]:
defined_experiment_combinations = [
    {
        "scaler": MinMaxScaler(),
        "encode": {"apply": True, "columns": ["categorical_col"]},
        "models": {"name": "RandomForest", "instance": RandomForestClassifier()}
    },
    {
        "scaler": None,
        "encode": {"apply": True, "columns": ["categorical_col"]},
        "models": {"name": "RandomForest", "instance": RandomForestClassifier()}
    },
    # {
    #     "scaler": StandardScaler(),
    #     "encode": {"apply": True, "columns": ["categorical_col"]},
    #     "models": {"name": "LightGBM", "instance": LGBMClassifier()}
    # },
    # {
    #     "scaler": None,
    #     "encode": {"apply": True, "columns": ["categorical_col"]},
    #     "models": {"name": "LightGBM", "instance": LGBMClassifier()}
    # },
]

In [21]:
# from ExperimentTracker2 import PhaseThreeExperimentTracker

# tracker = PhaseThreeExperimentTracker("Models with Scaler")

# # Load checkpoint file
# tracker.completed_runs

# # Run experiments with checkpointing
# tracker.run_experiments(
#     datasets=datasets,
#     experiment_combinations=defined_experiment_combinations,
#     X_test=X_test,
#     y_test=y_test,
#     numeric_columns=numeric_columns,
#     categorical_cols=categorical_cols
# )
